### Initialize SQL db

In [1]:

import sqlite3
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

#creat engine
engine = create_engine('sqlite:///./dbs/chinook.db')

#save the table names to a list
table_names = engine.table_names()
display(pd.DataFrame(table_names, columns= ['Table Names']))

,Table Names
0,albums
1,artists
2,customers
3,employees
4,genres
5,invoice_items
6,invoices
7,media_types
8,playlist_track
9,playlists


# 1
How many albums does the artist Led Zeppelin have?

In [37]:
#Open engine conncetion
con = engine.connect()
rs = con.execute("SELECT * FROM albums WHERE Artistid = ( \
                    SELECT ArtistID FROM artists \
                    WHERE Name LIKE '%Led%')")
df =pd.DataFrame(rs.fetchall(), columns = rs.keys())
display(df)
print("number of albums =",str(df.shape[0]))

,AlbumId,Title,ArtistId
0,30,BBC Sessions [Disc 1] [Live],22
1,44,Physical Graffiti [Disc 1],22
2,127,BBC Sessions [Disc 2] [Live],22
3,128,Coda,22
4,129,Houses Of The Holy,22
5,130,In Through The Out Door,22
6,131,IV,22
7,132,Led Zeppelin I,22
8,133,Led Zeppelin II,22
9,134,Led Zeppelin III,22


number of albums = 14


# 2
Create a list of album titles and the unit prices for the artist "Audioslave".

In [77]:
#fetch artists dt subset
rs = con.execute("SELECT * FROM artists \
                    WHERE Name LIKE '%slave%' \
                 ")
df =pd.DataFrame(rs.fetchall(), columns = rs.keys())
display(df)

,ArtistId,Name
0,8,Audioslave


In [148]:
con = engine.connect()
rs = con.execute("SELECT * \
                 FROM albums ALB \
                 LEFT JOIN artists ART \
                 ON ART.Artistid = ART.Artistid \
                 AND ART.Name = 'Audioslave' \
                 \
                 CROSS JOIN tracks T \
                 ON ALB.Albumid = T.Albumid \
                 \
                CROSS JOIN invoice_items INV \
                 ON INV.Trackid = T.Trackid \
                 ")
df =pd.DataFrame(rs.fetchall(), columns = rs.keys())
display(df)

# LEFT JOIN tracks T on ALB.Albumid = T.Albumid \
#                  LEFT JOIN invoice_items I on T.Trackid = I.Trackid \

,AlbumId,Title,ArtistId,ArtistId,Name,TrackId,Name,AlbumId,MediaTypeId,GenreId,Composer,Milliseconds,Bytes,UnitPrice,InvoiceLineId,InvoiceId,TrackId,UnitPrice,Quantity
0,1,For Those About To Rock We Salute You,1,8,Audioslave,1,For Those About To Rock (We Salute You),1,1,1,"Angus Young, Malcolm Young, Brian Johnson",343719,11170334,0.99,579,108,1,0.99,1
1,1,For Those About To Rock We Salute You,1,8,Audioslave,6,Put The Finger On You,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",205662,6713451,0.99,3,2,6,0.99,1
2,1,For Those About To Rock We Salute You,1,8,Audioslave,8,Inject The Venom,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",210834,6852860,0.99,4,2,8,0.99,1
3,1,For Those About To Rock We Salute You,1,8,Audioslave,8,Inject The Venom,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",210834,6852860,0.99,1155,214,8,0.99,1
4,1,For Those About To Rock We Salute You,1,8,Audioslave,9,Snowballed,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",203102,6599424,0.99,581,108,9,0.99,1
5,1,For Those About To Rock We Salute You,1,8,Audioslave,9,Snowballed,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",203102,6599424,0.99,1729,319,9,0.99,1
6,1,For Those About To Rock We Salute You,1,8,Audioslave,10,Evil Walks,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",263497,8611245,0.99,5,2,10,0.99,1
7,1,For Those About To Rock We Salute You,1,8,Audioslave,12,Breaking The Rules,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",263288,8596840,0.99,6,2,12,0.99,1
8,1,For Those About To Rock We Salute You,1,8,Audioslave,13,Night Of The Long Knives,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",205688,6706347,0.99,582,108,13,0.99,1
9,1,For Those About To Rock We Salute You,1,8,Audioslave,14,Spellbound,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",270863,8817038,0.99,1156,214,14,0.99,1


How many records are returned?

In [149]:
df.shape[0]

2240

# 3
Find the first and last name of any customer who does not have an invoice. Are there any customers returned from the query?

In [128]:
#fetch customers df
rs = con.execute("SELECT * \
                    FROM  customers C \
                    LEFT JOIN invoices I \
                    ON I.Customerid = C.Customerid \
                    AND I.invoiceid IS NULL \
                 ")
df =pd.DataFrame(rs.fetchall(), columns = rs.keys())
display(df.head())

,CustomerId,FirstName,LastName,Company,Address,City,State,Country,PostalCode,Phone,...,SupportRepId,InvoiceId,CustomerId,InvoiceDate,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,Total
0,1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,...,3,None,None,None,None,None,None,None,None,None
1,2,Leonie,Köhler,None,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,+49 0711 2842222,...,5,None,None,None,None,None,None,None,None,None
2,3,François,Tremblay,None,1498 rue Bélanger,Montréal,QC,Canada,H2G 1A7,+1 (514) 721-4711,...,3,None,None,None,None,None,None,None,None,None
3,4,Bjørn,Hansen,None,Ullevålsveien 14,Oslo,None,Norway,0171,+47 22 44 22 22,...,4,None,None,None,None,None,None,None,None,None
4,5,František,Wichterlová,JetBrains s.r.o.,Klanova 9/506,Prague,None,Czech Republic,14700,+420 2 4172 5555,...,4,None,None,None,None,None,None,None,None,None


# 4
Find the total price for each album.

In [137]:
#fetch customers df
rs = con.execute("SELECT SUM(UnitPrice) AS 'Total Price', * \
                    FROM albums AL \
                    LEFT JOIN tracks TR \
                    ON AL.Albumid = TR.Albumid \
                    GROUP BY TR.Albumid \
                 ")
df =pd.DataFrame(rs.fetchall(), columns = rs.keys())
display(df)

,Total Price,AlbumId,Title,ArtistId,TrackId,Name,AlbumId,MediaTypeId,GenreId,Composer,Milliseconds,Bytes,UnitPrice
0,9.90,1,For Those About To Rock We Salute You,1,14,Spellbound,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",270863,8817038,0.99
1,0.99,2,Balls to the Wall,2,2,Balls to the Wall,2,2,1,None,342562,5510424,0.99
2,2.97,3,Restless and Wild,2,5,Princess of the Dawn,3,2,1,Deaffy & R.A. Smith-Diesel,375418,6290521,0.99
3,7.92,4,Let There Be Rock,1,22,Whole Lotta Rosie,4,1,1,AC/DC,323761,10547154,0.99
4,14.85,5,Big Ones,3,37,Livin' On The Edge,5,1,1,"Steven Tyler, Joe Perry, Mark Hudson",381231,12374569,0.99
5,12.87,6,Jagged Little Pill,4,50,You Oughta Know (Alternate),6,1,1,Alanis Morissette & Glenn Ballard,491885,16008629,0.99
6,11.88,7,Facelift,5,62,Real Thing,7,1,1,"Jerry Cantrell, Layne Staley",243879,7937731,0.99
7,13.86,8,Warner 25 Anos,6,76,"Canta, Canta Mais",8,1,2,None,271856,8719426,0.99
8,7.92,9,Plays Metallica By Four Cellos,7,84,Welcome Home (Sanitarium),9,1,3,Apocalyptica,350197,11406431,0.99
9,13.86,10,Audioslave,8,98,The Last Remaining Light,10,1,1,Audioslave/Chris Cornell,317492,7622615,0.99


What is the total price for the album “Big Ones”?

In [141]:
df[df['Title']=='Big Ones']

,Total Price,AlbumId,Title,ArtistId,TrackId,Name,AlbumId,MediaTypeId,GenreId,Composer,Milliseconds,Bytes,UnitPrice
4,14.85,5,Big Ones,3,37,Livin' On The Edge,5,1,1,"Steven Tyler, Joe Perry, Mark Hudson",381231,12374569,0.99


# 5
How many records are created when you apply a Cartesian join to the invoice and invoice items table?

In [142]:
#fetch customers df
rs = con.execute("SELECT * \
                    FROM invoices \
                    CROSS JOIN invoice_items \
                 ")
df =pd.DataFrame(rs.fetchall(), columns = rs.keys())
display(df)

,InvoiceId,CustomerId,InvoiceDate,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,Total,InvoiceLineId,InvoiceId,TrackId,UnitPrice,Quantity
0,1,2,2009-01-01 00:00:00,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,1.98,1,1,2,0.99,1
1,1,2,2009-01-01 00:00:00,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,1.98,2,1,4,0.99,1
2,1,2,2009-01-01 00:00:00,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,1.98,3,2,6,0.99,1
3,1,2,2009-01-01 00:00:00,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,1.98,4,2,8,0.99,1
4,1,2,2009-01-01 00:00:00,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,1.98,5,2,10,0.99,1
5,1,2,2009-01-01 00:00:00,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,1.98,6,2,12,0.99,1
6,1,2,2009-01-01 00:00:00,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,1.98,7,3,16,0.99,1
7,1,2,2009-01-01 00:00:00,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,1.98,8,3,20,0.99,1
8,1,2,2009-01-01 00:00:00,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,1.98,9,3,24,0.99,1
9,1,2,2009-01-01 00:00:00,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,1.98,10,3,28,0.99,1


In [143]:
df.shape[0]

922880